<a href="https://colab.research.google.com/github/farhanfuad35/titanic/blob/main/Titanic_77_mean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Kaggle Environment and Loading the Dataset

In [ ]:
# Bash Scripts
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json   # 6: 4 + 2, Read & Write permission to myself

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# Downloading Datasets
! kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.1MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 9.42MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 60.5MB/s]


In [ ]:
# Importing Required Modules
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [ ]:
print(data_train.isnull().sum())
print(data_test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


# Data Imputation
## Age
For 'Age' I am going to use its mean value first to impute the data

In [ ]:
# Impute 'Age' column with its mean value
data_train_cleaned = data_train
data_test_cleaned = data_test

mean_age = data_train['Age'].mean()
print(mean_age)
data_train_cleaned = data_train.fillna(value={'Age': mean_age})

mean_age = data_test['Age'].mean()
print(mean_age)
data_test_cleaned = data_test_cleaned.fillna(value={'Age': mean_age})

29.69911764705882
30.272590361445783


In [ ]:
y = data_train_cleaned['Survived']

In [ ]:
# Convert Sex into Numerical Data
enc = OneHotEncoder(handle_unknown='ignore')
enc_data = pd.DataFrame(enc.fit_transform(data_train_cleaned[['Sex']]).toarray())
data_train_cleaned = data_train_cleaned.join(enc_data)
data_train_cleaned.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0,1
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0,1.0


In [ ]:
enc_data = pd.DataFrame(enc.fit_transform(data_test_cleaned[['Sex']]).toarray())
data_test_cleaned = data_test_cleaned.join(enc_data)
data_test_cleaned.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0,1
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0,1.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0,1.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0,0.0


In [ ]:
# Rename newly created numerical column names
data_train_cleaned = data_train_cleaned.rename(columns={0:'Male', 1:'Female'}, errors='raise')
data_test_cleaned = data_test_cleaned.rename(columns={0:'Male', 1:'Female'}, errors='raise')

print(data_train_cleaned.head())
print(data_test_cleaned.head())

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Male,Female
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0,1.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0,1.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1.0,0.0


In [ ]:
# Remove Unnecessary Columns
X_train = data_train_cleaned.drop(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], 1)
X_test = data_test_cleaned.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Fare', 'Cabin', 'Embarked'], 1)

In [ ]:
print(X_train.head())
print(X_test.head())

    Age  SibSp  Parch  Male  Female
0  22.0      1      0   0.0     1.0
1  38.0      1      0   1.0     0.0
2  26.0      0      0   1.0     0.0
3  35.0      1      0   1.0     0.0
4  35.0      0      0   0.0     1.0
    Age  SibSp  Parch  Male  Female
0  34.5      0      0   0.0     1.0
1  47.0      1      0   1.0     0.0
2  62.0      0      0   0.0     1.0
3  27.0      0      0   0.0     1.0
4  22.0      1      1   1.0     0.0


In [ ]:
# Train the Logistic Regression
logModel = LogisticRegression(max_iter=10000)
logModel.fit(X_train, y)

LogisticRegression(max_iter=10000)

In [ ]:
# Predict test dataset

predictions = logModel.predict(X_test)

output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [ ]:
# Submit via Kaggle API
! kaggle competitions submit titanic -f submission.csv -m "Kaggle API Submission"

100% 2.77k/2.77k [00:01<00:00, 1.52kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [ ]:
# See Submissions
# ! kaggle competitions submissions titanic

fileName        date                 description            status    publicScore  privateScore  
--------------  -------------------  ---------------------  --------  -----------  ------------  
submission.csv  2022-01-30 06:32:08  Kaggle API Submission  complete  0.77272      None          
submission.csv  2022-01-28 21:34:39  First Try. No tweak.   complete  0.62918      None          
